In [5]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import dataset_handler as dh
import testing_utils as tu
from termcolor import colored
import re
import numpy as np

In [2]:
priming_text_path = "data/priming_texts/gsm8k/codegen/func_eq_short.txt"
gsm8k_path = "data/grade-school-math/grade_school_math/data/train.jsonl"

current_dataset = dh.init_dataset_from_name(
        "gsm8k", primingtext_path=priming_text_path
    )

In [6]:
tu.set_all_seeds()

rand_indexes = np.random.randint(0, len(current_dataset.data), 100)
print(rand_indexes)

[4547 1364 2033 2409 2181 5082 2184 6995 5480 3775 2096 5997 5205 4924
 4211  458 4763  620 4365 6025 1085 6287 5469 3426 3602  270 7270 4445
 6584  777 4659 7108 5045 3893 5959 2875 6314 4506 6466 1894 3081 1765
 3453 2542 7249 3947 3855 1500 4443  491  575  598 7199 3092 2733 2581
 6856 7132 2076  735 2607 1418 2989 5122 4518 4295 6111 3195 1400 1241
 5060 3365 5483 4546 1574 5430 4054 4519 6778 5166 3718 2399 3232 4937
 7138 2055 5502 1417 2018 4838 6570 6163 4923 4400 1929 6672 6256  206
  118 4857]


In [9]:
test_data = [current_dataset.data[i] for i in rand_indexes]
test_data = [elem["question"] for elem in test_data]

In [10]:
test_data[0]

'Carol sells tickets for an exhibition. During three days she sold tickets worth $960. One ticket costs $4. How many tickets on average did she sell during one of these three days?'

In [11]:
import pickle

#pickle loads the model
clustering_model = pickle.load(open("clustering_model.pkl", "rb"))

In [15]:
embedder = SentenceTransformer('all-mpnet-base-v2')

# Corpus with example sentences
corpus = test_data
corpus_embeddings = embedder.encode(corpus)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


array([3], dtype=int32)

In [17]:
labels = list(clustering_model.predict(corpus_embeddings))

In [18]:
pkl_dir = "clustering_exp/pass_at_list/"

with open(f"{pkl_dir}cluster_0_pass_at_k.pkl", "rb") as f:
    cluster_0_pass_at_k = pickle.load(f)
with open(f"{pkl_dir}cluster_1_pass_at_k.pkl", "rb") as f:
    cluster_1_pass_at_k = pickle.load(f)
with open(f"{pkl_dir}cluster_2_pass_at_k.pkl", "rb") as f:
    cluster_2_pass_at_k = pickle.load(f)
with open(f"{pkl_dir}cluster_3_pass_at_k.pkl", "rb") as f:
    cluster_3_pass_at_k = pickle.load(f)

In [19]:
agg_pass_at_k = []
max_dict = {}
max_dict[0] = []
max_dict[1] = []
max_dict[2] = []
max_dict[3] = []


for i, label in enumerate(labels):
    cand_list = [
        cluster_0_pass_at_k[i],
        cluster_1_pass_at_k[i],
        cluster_2_pass_at_k[i],
        cluster_3_pass_at_k[i],
    ]
    max_cand = cand_list[label]
    max_dict[cand_list.index(max_cand)].append(i)
    agg_pass_at_k.append(max_cand)

In [22]:
print(f"\nAggregate prompts - Pass@{3} = {np.mean(np.array(agg_pass_at_k))}\n")
print(f"cluster_0: {len(max_dict[0])}")
print(f"cluster_1: {len(max_dict[1])}")
print(f"cluster_2: {len(max_dict[2])}")
print(f"cluster_3: {len(max_dict[3])}")


Aggregate prompts - Pass@3 = 0.38299999999999995

cluster_0: 65
cluster_1: 23
cluster_2: 5
cluster_3: 7


In [23]:
with open(f"{pkl_dir}general_pass_at_k.pkl", "rb") as f:
    general_pass_at_k = pickle.load(f)

print(f"General prompts - Pass@{3} = {np.mean(np.array(general_pass_at_k))}")

General prompts - Pass@3 = 0.355


In [27]:
len(list(np.nonzero(np.array(agg_pass_at_k)))[0])

46